## Bienvenido a este arenero donde juguetear

Adjunto celdas de codigo con funciones directamnete para importar y probar.

**Importante revisar los requirements contengan mismas versiones y paquetes instaladas**

langchain==1.1.3
langgraph==1.0.4
langchain-openai==1.1.3
pydantic==2.12.5
requests==2.32.5
python-dotenv==1.2.1

### Funcion Busqueda Google Places API

In [1]:
from backend_google_places import PlaceSearchPayload, places_text_search

# ---------------- Prueba interactiva ----------------
payload = PlaceSearchPayload(
    query="Pizzeria",
    location="Calle Galileo 94, Navalcarnero",
    # radius=1500,
    # price_level=2,
    # max_travel_time=10,
    travel_mode="driving",
    # extras=["vegano"]
)

resultados = places_text_search(payload)
# print(f"Total resultados: {resultados['total_results']}")
for r in resultados:
    print(r)

⚠️  Autocomplete falló, intentando con Geocoding API...
✅ Geocodificado (Geocoding API) 'Calle Galileo 94, Navalcarnero' → 40.28378560000001,-4.0076355
   Dirección: C. de Galileo, 94, 28600 Navalcarnero, Madrid, Spain
{'name': 'Pizzería Tío Miguel', 'address': 'Calle de Jacinto González, 65, 28600 Navalcarnero, Madrid, Spain', 'place_id': 'ChIJI01AeVqUQQ0RuuhuD9AJXk4', 'types': ['establishment', 'food', 'meal_delivery', 'point_of_interest'], 'rating': 4.4, 'user_ratings_total': 1984, 'price_level': 1, 'location': {'lat': 40.2873028, 'lng': -4.0181846}, 'neighborhood': 'Navalcarnero', 'phone': '918 11 38 02', 'website': 'https://www.pizzeriatiomiguel.es/es/', 'opening_hours': {'open_now': False, 'periods': [{'close': {'day': 1, 'time': '0005'}, 'open': {'day': 0, 'time': '1300'}}, {'close': {'day': 3, 'time': '0005'}, 'open': {'day': 2, 'time': '1800'}}, {'close': {'day': 4, 'time': '0005'}, 'open': {'day': 3, 'time': '1800'}}, {'close': {'day': 5, 'time': '0005'}, 'open': {'day': 4, '

In [2]:
# ---------------- Prueba interactiva ----------------
payload = PlaceSearchPayload(
    query="Pizzeria",
    location="Navalcarnero",
    # radius=1500,
    # price_level=2,
    # max_travel_time=10,
    # travel_mode="driving",
    # extras=["vegano"]
)

resultados = places_text_search(payload)
# print(f"Total resultados: {resultados['total_results']}")
for r in resultados:
    print(r)

✅ Geocodificado 'Navalcarnero' → 40.285599,-4.0166526
   Dirección: 28600 Navalcarnero, Madrid, Spain
{'name': 'Pizzería Tío Miguel', 'address': 'Calle de Jacinto González, 65, 28600 Navalcarnero, Madrid, Spain', 'place_id': 'ChIJI01AeVqUQQ0RuuhuD9AJXk4', 'types': ['establishment', 'food', 'meal_delivery', 'point_of_interest'], 'rating': 4.4, 'user_ratings_total': 1984, 'price_level': 1, 'location': {'lat': 40.2873028, 'lng': -4.0181846}, 'neighborhood': 'Navalcarnero', 'phone': '918 11 38 02', 'website': 'https://www.pizzeriatiomiguel.es/es/', 'opening_hours': {'open_now': False, 'periods': [{'close': {'day': 1, 'time': '0005'}, 'open': {'day': 0, 'time': '1300'}}, {'close': {'day': 3, 'time': '0005'}, 'open': {'day': 2, 'time': '1800'}}, {'close': {'day': 4, 'time': '0005'}, 'open': {'day': 3, 'time': '1800'}}, {'close': {'day': 5, 'time': '0005'}, 'open': {'day': 4, 'time': '1800'}}, {'close': {'day': 6, 'time': '0005'}, 'open': {'day': 5, 'time': '1800'}}, {'close': {'day': 0, 'tim

In [3]:
resultados

[{'name': 'Pizzería Tío Miguel',
  'address': 'Calle de Jacinto González, 65, 28600 Navalcarnero, Madrid, Spain',
  'place_id': 'ChIJI01AeVqUQQ0RuuhuD9AJXk4',
  'types': ['establishment', 'food', 'meal_delivery', 'point_of_interest'],
  'rating': 4.4,
  'user_ratings_total': 1984,
  'price_level': 1,
  'location': {'lat': 40.2873028, 'lng': -4.0181846},
  'neighborhood': 'Navalcarnero',
  'phone': '918 11 38 02',
  'website': 'https://www.pizzeriatiomiguel.es/es/',
  'opening_hours': {'open_now': False,
   'periods': [{'close': {'day': 1, 'time': '0005'},
     'open': {'day': 0, 'time': '1300'}},
    {'close': {'day': 3, 'time': '0005'}, 'open': {'day': 2, 'time': '1800'}},
    {'close': {'day': 4, 'time': '0005'}, 'open': {'day': 3, 'time': '1800'}},
    {'close': {'day': 5, 'time': '0005'}, 'open': {'day': 4, 'time': '1800'}},
    {'close': {'day': 6, 'time': '0005'}, 'open': {'day': 5, 'time': '1800'}},
    {'close': {'day': 0, 'time': '0005'}, 'open': {'day': 6, 'time': '1300'}}],


In [ ]:
print(resultados)

### Agente input Streamlit + busqueda lugares con Google Places API

In [2]:
# from langchain.agents import create_agent
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import Optional
import os
from dotenv import load_dotenv, find_dotenv

from backend_google_places import places_text_search, PlaceSearchPayload

load_dotenv(find_dotenv(), override=True)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
llm_openai = ChatOpenAI(model="gpt-4o-mini", temperature=0, openai_api_key=OPENAI_API_KEY)

# Define el schema de entrada CLARAMENTE
class PlaceSearchInput(BaseModel):
    """Input para buscar lugares con Google Places API"""
    query: str = Field(description="Tipo de lugar a buscar (ej: 'pizzeria', 'restaurante italiano')")
    location: str = Field(description="Ubicación base para la búsqueda (dirección completa o ciudad)")
    radius: Optional[int] = Field(None, description="Radio de búsqueda en metros")
    max_travel_time: Optional[int] = Field(None, description="Tiempo máximo de viaje en minutos")
    travel_mode: Optional[str] = Field("walking", description="Modo de viaje: walking, driving, transit, bicycling")
    price_level: Optional[int] = Field(None, description="Nivel de precio: 1=económico, 2=moderado, 3=caro, 4=lujo")
    extras: Optional[str] = Field(None, description="Características adicionales deseadas (ej: 'terraza', 'wifi')")



@tool(args_schema=PlaceSearchInput)
def search_places(
    query: str,
    location: str,
    radius: Optional[int] = None,
    max_travel_time: Optional[int] = None,
    travel_mode: str = "walking",
    price_level: Optional[int] = None,
    extras: Optional[str] = None
) -> str:
    """
    Busca lugares usando Google Places API.
    
    Retorna una lista de lugares que coinciden con los criterios de búsqueda.
    Usa esta herramienta cuando necesites encontrar restaurantes, tiendas u otros lugares.
    """
    from types import SimpleNamespace
    
    # ✅ CALCULAR RADIUS AUTOMÁTICO ANTES DE CREAR EL PAYLOAD
    if radius is None:
        if max_travel_time is not None:
            # Caso: tiene max_travel_time pero no radius → calcular radius
            if travel_mode == "driving":
                radius = max_travel_time * 833  # ~50 km/h → 833 m/min
            elif travel_mode == "bicycling":
                radius = max_travel_time * 250  # ~15 km/h → 250 m/min
            else:  # walking o transit
                radius = max_travel_time * 80   # ~5 km/h → 80 m/min
            print(f"  🔧 Radius calculado: {radius}m para {max_travel_time} min en {travel_mode}")
        else:
            # Caso: ni radius ni max_travel_time → usar default
            radius = 5000  # Default 5km
            print(f"  🔧 Usando radius por defecto: {radius}m")
    else:
        # Caso: radius ya proporcionado explícitamente
        print(f"  ℹ️ Usando radius proporcionado: {radius}m")
    
    # DEBUG: Imprime lo que recibe
    print(f"\n🔍 DEBUG SEARCH_PLACES:")
    print(f"  query: {query}")
    print(f"  location: {location}")
    print(f"  radius: {radius}")  # ← Ahora debería tener valor
    print(f"  max_travel_time: {max_travel_time}")
    print(f"  travel_mode: {travel_mode}")
    print(f"  price_level: {price_level}")
    print(f"  extras: {extras}")
    
    # Crear el payload CON TODOS LOS ATRIBUTOS
    payload = SimpleNamespace(
        query=query,
        location=location,
        radius=radius,  # ← Ahora tiene valor calculado
        max_travel_time=max_travel_time,
        travel_mode=travel_mode if travel_mode else "walking",
        price_level=price_level,
        extras=extras
    )
    
    # Llamar a tu función real
    try:
        resultados = places_text_search(payload)
        print(f"  ✅ Resultados obtenidos: {len(resultados) if resultados else 0}")
    except Exception as e:
        print(f"  ❌ Error en places_text_search: {e}")
        import traceback
        traceback.print_exc()
        return f"Error al buscar lugares: {str(e)}"
    
    # Formatear resultados (YA NO NECESITAS EL REINTENTO)
    if not resultados or len(resultados) == 0:
        return "No se encontraron resultados para la búsqueda."
    
    output = f"Se encontraron {len(resultados)} lugares:\n\n"
    for i, r in enumerate(resultados[:5], 1):  # Limitar a 5 para no saturar
        output += f"{i}. {r['name']}\n"
        output += f"   📍 {r['address']}\n"
        output += f"   ⭐ Rating: {r.get('rating', 'N/A')} ({r.get('user_ratings_total', 0)} reseñas)\n"
        if r.get('price_level'):
            output += f"   💰 Precio: {'€' * r['price_level']}\n"
        if r.get('phone'):
            output += f"   📞 {r['phone']}\n"
        if r.get('website'):
            output += f"   🌐 {r['website']}\n"
        output += "\n"
    
    return output

# Crear el prompt personalizado
prompt = ChatPromptTemplate.from_messages([
    ("system", """Eres un asistente experto en encontrar lugares y restaurantes.

Cuando el usuario te pida buscar un lugar:
1. Identifica el tipo de lugar (query) y la ubicación (location)
2. Extrae características adicionales si las menciona (terraza, precio, etc.)
3. USA LA HERRAMIENTA search_places con los parámetros apropiados
4. Presenta los resultados de forma clara y amigable

IMPORTANTE: 
- Siempre debes LLAMAR a la herramienta search_places para buscar lugares reales
- No inventes restaurantes ni lugares
- Si el usuario menciona "asequible" o "económico", usa price_level=2
- Si menciona "terraza", inclúyelo en el campo extras
- Si menciona tiempo de viaje en coche, usa travel_mode="driving"
- Si menciona "10 minutos", usa max_travel_time=10"""),
    ("placeholder", "{messages}"),
])

# Crear el agente CON EL PROMPT
agent = create_react_agent(
    model=llm_openai,
    tools=[search_places],
    prompt=prompt
)

# Probar
input_text = "Busca un restaurante asiatico en Navalcarnero, a menos de 20 minutos en coche desde Calle Galileo 94"

resultado = agent.invoke(
    {"messages": [{"role": "user", "content": input_text}]}
)

print("\n=== RESPUESTA FINAL ===")
print(resultado['messages'][-1].content)

# Para ver el proceso completo (debug)
print("\n\n=== PROCESO COMPLETO (DEBUG) ===")
for i, msg in enumerate(resultado['messages']):
    print(f"\n--- Mensaje {i+1}: {msg.__class__.__name__} ---")
    if hasattr(msg, 'content'):
        print(msg.content)
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        print("TOOL CALLS:", msg.tool_calls)

C:\Users\Alejandro\AppData\Local\Temp\ipykernel_8176\2074682302.py:139: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(


  🔧 Radius calculado: 16660m para 20 min en driving

🔍 DEBUG SEARCH_PLACES:
  query: restaurante asiático
  location: Navalcarnero
  radius: 16660
  max_travel_time: 20
  travel_mode: driving
  price_level: None
  extras: None
  ✅ Resultados obtenidos: 0

=== RESPUESTA FINAL ===
No encontré restaurantes asiáticos en Navalcarnero que estén a menos de 20 minutos en coche desde Calle Galileo 94. Si deseas, puedo buscar en una ubicación diferente o con otros criterios. ¿Te gustaría intentar con otra opción?


=== PROCESO COMPLETO (DEBUG) ===

--- Mensaje 1: HumanMessage ---
Busca un restaurante asiatico en Navalcarnero, a menos de 20 minutos en coche desde Calle Galileo 94

--- Mensaje 2: AIMessage ---

TOOL CALLS: [{'name': 'search_places', 'args': {'query': 'restaurante asiático', 'location': 'Navalcarnero', 'max_travel_time': 20, 'travel_mode': 'driving'}, 'id': 'call_dYInc0PIuSaYLvW5OWnQlbLH', 'type': 'tool_call'}]

--- Mensaje 3: ToolMessage ---
No se encontraron resultados para la bú